In [1]:
from pyfirmata import Arduino

In [3]:
pin_diode = 13
pin_TTL = 6

class MCSArduino(Arduino):
    def __init__(self, *args, **kwargs):
        self.last_cmd = False  # False - Arduino LOW, True - Arduino HIGH
        super(MCSArduino, self).__init__(*args, **kwargs)
        
    def start_or_stop(self):
        self.last_cmd = not self.last_cmd
        self.digital[pin_diode].write(self.last_cmd)
        self.digital[pin_TTL].write(self.last_cmd)

class FakeArduino():
    def start_or_stop(self):
        print("Fake Arduino - sending a TTL pulse")
        
    def exit(self):
        print("Fake Arduino - exiting...")

In [ ]:
# Initialize REAL Arduino if connected
if cfg['MCSArduinoPort'] == 'fake':
    board = FakeArduino()
else:
    board = MCSArduino(cfg['MCSArduinoPort'])  # Windows - 'COM10', Linux - '/dev/ttyACM0', check /dev/tty*